In [1]:
run_checks = False

### Overview
This notebook works on the IEEE-CIS Fraud Detection competition. Here I build a simple XGBoost model based on a balanced dataset.

### Lessons:

. keep the categorical variables as single items

. Use a high max_depth for xgboost (maybe 40)


### Ideas to try:

. train divergence of expected value (eg. for TransactionAmt and distance based on the non-fraud subset (not all subset as in the case now)

. try using a temporal approach to CV

In [2]:
# all imports necessary for this notebook
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import gc
import copy
import missingno as msno 
import xgboost
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split 
from sklearn.metrics import roc_auc_score, r2_score

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/master-df-time-adjusted-top-100csv/master_df_time_adjusted_top_100.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/master-df-time-adjusted-top-300csv/master_df_time_adjusted_top_300.csv
/kaggle/input/master-df-time-adjusted-top-200csv/master_df_time_adjusted_top_200.csv
/kaggle/input/ieee-preprocessed/master_df_top_all.csv
/kaggle/input/ieee-preprocessed/master_df_top_300.csv
/kaggle/input/ieee-preprocessed/master_df_top_100.csv
/kaggle/input/ieee-preprocessed/master_df_top_200.csv


In [3]:
# Helpers
    
def seed_everything(seed=0):
    '''Seed to make all processes deterministic '''
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
def drop_correlated_cols(df, threshold, cols_to_keep, sample_frac = 1):
    '''Drops one of two dataframe's columns whose pairwise pearson's correlation is above the provided threshold'''
    if sample_frac != 1:
        dataset = df.sample(frac = sample_frac).copy()
    else:
        dataset = df
        
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        if corr_matrix.columns[i] in col_corr:
            continue
        for j in range(i):
            if corr_matrix.columns[j] in cols_to_keep:
                continue
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
    del dataset
    gc.collect()
    df.drop(columns = col_corr, inplace = True)

def calc_feature_difference(df, feature_name, indep_features, min_r2 = 0.1, min_r2_improv = 0, frac1 = 0.1,
                              max_depth_start = 2, max_depth_step = 4):
    
    from copy import deepcopy
    
    print("Feature name %s" %feature_name)
    #print("Indep_features %s" %indep_features)
    
    is_imrpoving = True
    curr_max_depth = max_depth_start
    best_r2 = float("-inf")
    clf_best = np.nan
    
    while is_imrpoving:
        clf = XGBRegressor(max_depth = curr_max_depth)

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        clf.fit(df.loc[rand_sample_indeces, indep_features], df.loc[rand_sample_indeces, feature_name]) 

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        
        pred_y = clf.predict(df.loc[rand_sample_indeces, indep_features])
        r2Score = r2_score(df.loc[rand_sample_indeces, feature_name], pred_y)
        print("%d, R2 score %.4f" % (curr_max_depth, r2Score))
        
        curr_max_depth = curr_max_depth + max_depth_step
        
        if r2Score > best_r2:
            best_r2 = r2Score
            clf_best = deepcopy(clf)
        if r2Score < best_r2 + (best_r2 * min_r2_improv) or (curr_max_depth > max_depth_start * max_depth_step and best_r2 < min_r2 / 2):
            is_imrpoving = False

    print("The best R2 score of %.4f" % ( best_r2))
    
    if best_r2 > min_r2:
        pred_feature = clf_best.predict(df.loc[:, indep_features])
        return (df[feature_name] - pred_feature)
    else:
        return df[feature_name]

In [4]:
seed_everything()
pd.set_option('display.max_columns', 500)

In [5]:
master_df = pd.read_csv('/kaggle/input/master-df-time-adjusted-top-300csv/master_df_time_adjusted_top_300.csv')
master_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D10,D11,D12,D13,D14,D15,M2,M3,M4,M5,M6,M7,M8,M9,V2,V3,V4,V5,V6,V7,V10,V12,V13,V19,V20,V21,V23,V24,V25,V29,V34,V35,V36,V37,V38,V39,V40,V44,V45,V47,V48,V49,V51,V52,V53,V54,V55,V56,V60,V61,V62,V64,V66,V67,V69,V70,V74,V75,V76,V77,V78,V79,V81,V82,V83,V86,V87,V90,V91,V99,V100,V104,V105,V109,V110,V112,V115,V116,V117,V122,V124,V126,V127,V128,V129,V130,V131,V135,V136,V137,V139,V141,V144,V145,V146,V149,V150,V151,V152,V154,V160,V161,V162,V164,V165,V166,V170,V171,V173,V176,V187,V188,V189,V191,V192,V194,V201,V202,V203,V206,V208,V209,V211,V214,V218,V221,V222,V223,V224,V225,V226,V229,V230,V234,V236,V243,V244,V245,V247,V248,V249,V253,V254,V257,V258,V263,V264,V265,V266,V267,V269,V270,V271,V272,V273,V281,V282,V283,V285,V286,V287,V288,V290,V291,V292,V296,V298,V301,V302,V303,V304,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V317,V318,V319,V320,V323,V331,V332,V333,V337,id_01,id_02,id_03,id_05,id_06,id_07,id_09,id_13,id_14,id_16,id_17,id_18,id_19,id_20,id_25,id_26,id_34,id_36,id_38,DeviceType,DeviceInfo,is_train_df,P_emaildomain_2,P_emaildomain_3,P_emaildomain_4,id_31_chrome,id_31_for,id_31_android,id_31_safari,id_31_generic,id_31_edge,id_31_ie,id_31_chrome_version_newness,id_31_safari_version_newness,id_30_android,id_30_windows,id_33_resolution,TransactionAmt_decimal,TransactionAmt_decimal_length,weekday,hours,id_18_15.0,card3_infrequent_category,R_emaildomain_2_com,id_14_-420.0,id_14_-360.0,id_14_60.0,M4_M0,M4_M1,M8_F,id_13_19.0,id_13_49.0,id_13_52.0,R_emaildomain_anonymous.com,R_emaildomain_gmail.com,R_emaildomain_hotmail.com,P_emaildomain_4_com,id_19_100.0,id_19_193.0,id_19_266.0,id_19_271.0,id_19_312.0,id_19_321.0,id_19_410.0,id_19_427.0,id_19_infrequent_category,M6_F,M3_F,M5_F,id_31_firefox_False,M2_F,P_emaildomain_anonymous.com,P_emaildomain_aol.com,P_emaildomain_bellsouth.net,P_emaildomain_comcast.net,P_emaildomain_cox.net,P_emaildomain_gmail.com,P_emaildomain_hotmail.com,P_emaildomain_icloud.com,P_emaildomain_live.com,P_emaildomain_live.com.mx,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_optonline.net,P_emaildomain_outlook.com,P_emaildomain_verizon.net,P_emaildomain_yahoo.com,P_emaildomain_infrequent_category,M9_F,id_20_222.0,id_20_325.0,id_20_333.0,id_20_401.0,id_20_500.0,id_20_507.0,id_20_533.0,id_20_549.0,id_20_595.0,id_20_597.0,id_20_612.0,id_20_infrequent_category,id_38_F,M7_F,P_emaildomain_1_hotmail,P_emaildomain_1_outlook,P_emaildomain_1_infrequent_category,id_17_100.0,id_32_24.0,DeviceInfo_MacOS,DeviceInfo_SM-J700M Build/MMB29K,DeviceInfo_Trident/7.0,DeviceInfo_Windows,DeviceInfo_infrequent_category
0,2987000,0.0,0,68.5,4,12696,-1,50,1,42,1,215,77,19.0,NaN,NaN,NaN,0.452748,-56.367245,-0.207838,0.0,-3.934973,1.0,0.424639,-4.162749,-0.019222,3.993362,-10.398003,-54.096107,NaN,13.0,NaN,NaN,NaN,NaN,NaN,-78.381050,13.0,NaN,NaN,NaN,-66.240028,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.455908,0.253646,0.115072,1.0,0.003947,1.0,1.0,0.045288,-0.828852,0.002764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.430471,0.233582,1.0,-0.065734,0.000217,0.144780,1.0,0.000742,1.0,1.0,-1.038489,-0.588924,-0.004857,0.232415,0.276390,-0.416617,-0.058481,0.003955,-0.027670,-0.785578,0.0,1.0,-0.205193,-0.362073,-0.800531,-0.271201,-0.131641,-0.041035,-0.967687,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-54.354218,-6127.442383,-87.003616,0.0,0.0,0.0,0.0,-38.625824,-26.290359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009348,1.0,-0.340087,-0.622000,0.0,-0.820475,0.0,-0.842723,-0.346300,-0.106636,-0.276840,0.175033,0.0,-0.001258,-0.022594,0.002590,-25.646389,-133.207397,-188.566223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1

In [6]:
for col in master_df.select_dtypes(exclude='number').columns:
    master_df[col] = master_df[col].astype('category').cat.codes

In [7]:
'''
length_ones = len(master_df[master_df['isFraud']==1])
train_balanced = pd.concat([master_df[master_df['isFraud']==1], (master_df[master_df['isFraud']==0]).sample(length_ones)], axis=0)

#train_balanced = train_balanced.sample(10000)


X_train, X_test, y_train, y_test = train_test_split(
    train_balanced.drop(columns=['isFraud', 'TransactionID', 'TransactionDT']), train_balanced['isFraud'], 
    test_size=1/4, stratify =train_balanced['isFraud'],  random_state=0)

print(X_train.shape)
print(X_test.shape)

clf = XGBClassifier(max_depth=5, n_estimators=100, verbosity=1)
clf.fit(X_train, y_train)
pred_prob = clf.predict_proba(X_test)
pred_prob[:, 1]
roc_score = roc_auc_score(y_test, pred_prob[:, 1])
print("roc_auc score %.4f" % roc_score)
xgboost.plot_importance(clf, max_num_features=20, importance_type='gain')
xgboost.plot_importance(clf, max_num_features=20, importance_type='weight')
'''

'\nlength_ones = len(master_df[master_df[\'isFraud\']==1])\ntrain_balanced = pd.concat([master_df[master_df[\'isFraud\']==1], (master_df[master_df[\'isFraud\']==0]).sample(length_ones)], axis=0)\n\n#train_balanced = train_balanced.sample(10000)\n\n\nX_train, X_test, y_train, y_test = train_test_split(\n    train_balanced.drop(columns=[\'isFraud\', \'TransactionID\', \'TransactionDT\']), train_balanced[\'isFraud\'], \n    test_size=1/4, stratify =train_balanced[\'isFraud\'],  random_state=0)\n\nprint(X_train.shape)\nprint(X_test.shape)\n\nclf = XGBClassifier(max_depth=5, n_estimators=100, verbosity=1)\nclf.fit(X_train, y_train)\npred_prob = clf.predict_proba(X_test)\npred_prob[:, 1]\nroc_score = roc_auc_score(y_test, pred_prob[:, 1])\nprint("roc_auc score %.4f" % roc_score)\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'gain\')\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'weight\')\n'

In [8]:
train_balanced = master_df[master_df['isFraud'].notnull()]

temp_list_to_drop = []
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

print(train_balanced.shape)


clf = XGBClassifier(max_depth=50)
clf.fit(train_balanced.drop(columns=temp_list_to_drop), train_balanced['isFraud'])

(590540, 339)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [9]:
gc.collect()

19

In [10]:
# prepare submission
temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])
temp_list_to_include = list(set(master_df.columns).difference(set(temp_list_to_drop)))

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])

temp_list_to_include =  list(train_balanced.drop(columns=temp_list_to_drop).columns)

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]

len_master_df = len(master_df)

print(counter_from)
print(len_master_df)
print('start!!')
while counter_from < len_master_df:
    print(counter_from)
    counter_to = counter_from + 10000
    pred = pd.DataFrame()
    #print(len(master_df['isFraud'].loc[counter_from:counter_to]))
    #print(len(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]]))
    master_df['isFraud'].loc[counter_from:counter_to] =  clf.predict_proba(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]])[:, 1]
    
    counter_from += 10000
    gc.collect()
#print(temp_list_to_include)

590540
1097231
start!!
590540
600540
610540
620540
630540
640540
650540
660540
670540
680540
690540
700540
710540
720540
730540
740540
750540
760540
770540
780540
790540
800540
810540
820540
830540
840540
850540
860540
870540
880540
890540
900540
910540
920540
930540
940540
950540
960540
970540
980540
990540
1000540
1010540
1020540
1030540
1040540
1050540
1060540
1070540
1080540
1090540


In [11]:
#sample_submission.head()

In [12]:
counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]
submission = pd.DataFrame(master_df[['TransactionID', 'isFraud']].loc[counter_from:]).reset_index(drop = True)
submission.head()

,TransactionID,isFraud
0,3663549,0.000320
1,3663550,0.000642
2,3663551,0.001093
3,3663552,0.000464
4,3663553,0.000517


In [13]:
submission.describe()

,TransactionID,isFraud
count,5.066910e+05,506691.000000
mean,3.916894e+06,0.018706
std,1.462692e+05,0.106231
min,3.663549e+06,0.000061
25%,3.790222e+06,0.000603
50%,3.916894e+06,0.001357
75%,4.043566e+06,0.003735
max,4.170239e+06,0.999704


In [14]:
submission.to_csv('submission.csv', index=False)